In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
import pydicom
from pyftpp import CT, Dose, Structure, StructureSet
from pyftpp import DVH
from pyftpp import PyFTPPConfigurator, PyFTPPRunner
from pyftpp import CTPlotter, DvhPlotter
from pyftpp.dicom import export_to_dicom

# Config

In [ ]:
output_dir = os.path.abspath('example')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
ct_path = f'{output_dir}/waterblock_ct.dat'

# Save the water phantom data in the tests/ directory.
phantom_dir = 'tests/data/water_phantom'

if not os.path.exists(phantom_dir):
    os.makedirs(phantom_dir)

# Generate the water phantom

In [ ]:
# CT.
# Usually we would just give the path to an existing CT image, but here we
# have to write it first to disk.
# The data is a simple water phantom in air, with a high-density cube in the middle.
data = np.full((100, 100, 40), -1000.)
data[20:80, 20:80, 20:30] = 0.
data[40:60, 40:60, 22:28] = 3000
spacing = np.array([0.1, 0.1, 0.2])

ct = CT(data, spacing)
ct.save(ct_path)
ct.save(f'{phantom_dir}/ct.dat')

In [ ]:
# To run a plan evaluation, we just need to pass a path to the CT on disk.
# We can load a CT like this (useful for visualisation):
# ct = CT.load(ct_path)

In [ ]:
# The contours are given as 3D points in physical coordinates.
# The data format is required by the treatment planning system.
def build_oar_contour(spcaing):
    '''
    Build the contour for an Organ At Risk (OAR).

    It is a small cube in a corner of the phantom.
    '''
    slices = []
    for z in range(22, 28+1):
        slice = [
            {
                'x': 40*spacing[0],
                'y': 40*spacing[1],
                'z': z*spacing[2],
            },
            {
                'x': 60*spacing[0],
                'y': 40*spacing[1],
                'z': z*spacing[2],
            },
            {
                'x': 60*spacing[0],
                'y': 60*spacing[1],
                'z': z*spacing[2],
            },
            {
                'x': 40*spacing[0],
                'y': 60*spacing[1],
                'z': z*spacing[2],
            },
        ]
        slice = {'points': slice}
        slices.append(slice)
    return slices

In [ ]:
def build_target_contour(spacing):
    target_slices = []
    for z in range(20, 30+1):
        slice = [
            {
                'x': 20*spacing[0],
                'y': 20*spacing[1],
                'z': z*spacing[2],
            },
            {
                'x': 80*spacing[0],
                'y': 20*spacing[1],
                'z': z*spacing[2],
            },
            {
                'x': 80*spacing[0],
                'y': 80*spacing[1],
                'z': z*spacing[2],
            },
            {
                'x': 20*spacing[0],
                'y': 80*spacing[1],
                'z': z*spacing[2],
            },
        ]
        slice = {'points': slice}
        target_slices.append(slice)

    return target_slices

In [ ]:
def list_of_slices_to_numpy(slices):
    '''
    Convert the FTPP slice format to a numpy array.
    Each row of the array is a contour point.
    '''
    points = []
    for s in slices:
        for p in s['points']:
            points.append([p['x'], p['y'], p['z']])
    return np.array(points)

In [ ]:
# OAR contour.
oar_slices = build_oar_contour(spacing)
oar_points = list_of_slices_to_numpy(oar_slices)

# Target contour.
target_slices = build_target_contour(spacing)
target_points = list_of_slices_to_numpy(target_slices)

In [ ]:
target_structure = Structure('water_target', target_points, ct.grid)
oar_structure = Structure('my_OAR', oar_points, ct.grid)

# Save the target and the OAR contours.

In [ ]:
np.save(f'{phantom_dir}/target.npy', target_points)
np.save(f'{phantom_dir}/oar.npy', oar_points)

# Configure the optimizer

In [ ]:
# Target prescription.
target_dose = 1.8
target_ID = 0
oar_ID = 0
constraint_ID = 0

# Field and optimization specification.
# We only use a single field and a single constraint.
conf = PyFTPPConfigurator(ct_path, target_dose, 0.9*target_dose)
conf.set_target(target_ID, target_structure)
conf.add_field(target_ID,
               gantry_angle=0,
               couch_angle=0)
conf.add_field(target_ID,
               gantry_angle=90,
               couch_angle=0)
conf.add_field(target_ID,
               gantry_angle=180,
               couch_angle=0)
conf.add_field(target_ID,
               gantry_angle=270,
               couch_angle=0)
conf.add_constraint(oar_ID,
                    constraint_ID,
                    oar_structure,
                    # We think that fulfilling this constraint is 0.5 times as important as covering the target.
                    importance=2,
                    # We want the dose in the OAR to be less than half the target dose.
                    dose=0.,
                    # Only relevant for DVH constraints.
                    volume=0.,
                    # type is either DOSE_VOLUME or MEAN.
                    type='DOSE_VOLUME')

# Run the optimiser

In [ ]:
runner = PyFTPPRunner(output_dir)
runner.add_configuration(conf, 'my_run')

print('Starting spot optimisation and dose calculation...')
result_directories = runner.run_all()
print('Done.')

In [ ]:
result_directories = {
    'my_run': 'example/my_run'
}

# Load the results

In [ ]:
dose = Dose.load(f'{result_directories["my_run"]}/result_dose.dat')

# Save everything to DICOM

In [ ]:
patient_ID = 'cube water phantom'
study_instance_UID = pydicom.uid.generate_uid(entropy_srcs=[patient_ID])
structureset = StructureSet([oar_structure, target_structure], ct)
dicom_dir = f'{phantom_dir}/DICOM'

if not os.path.exists(dicom_dir):
    os.makedirs(dicom_dir)

export_to_dicom(
    ct,
    structureset,
    dicom_dir,
    study_instance_UID,
    patient_ID,
    dose_distributions={'my_dose': dose},
)

# Plot the CT

In [ ]:
%matplotlib widget
red = (255, 0, 0)
green = (0, 255, 0)

plotter = CTPlotter(ct, dose, target_dose, crosshair_visible=False);
plotter.add_structure(target_structure, red)
plotter.add_structure(oar_structure, green)
plotter

# Plot the dose-volume-histograms (DVH)

In [ ]:
dvh = DVH(dose, target_structure.mask)
oar_dvh = DVH(dose, oar_structure.mask)

In [ ]:
%matplotlib inline
plotter = DvhPlotter(target_dose, 'my_plot')
plotter.add_curve('target_curve', dvh, (1, 0, 0))
plotter.add_curve('oar_curve', oar_dvh, (0, 0.5, 0))
plotter.plot()

# Calculating Dx% and Vx%

In [ ]:
target_V95 = dvh.V(0.95 * target_dose)
oar_D2 = dvh.D(0.02)

In [ ]:
q = np.arange(0, 1.50, 0.01)
oar_dvh = DVH(dose, target_structure.mask)
target_dvh = DVH(dose, oar_structure.mask)

oar_volumes = oar_dvh.V(q * target_dose)
target_volumes = target_dvh.V(q * target_dose)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()

ax.plot(q*100, target_volumes*100, label='target')
ax.plot(q*100, oar_volumes*100, label='OAR')

ax.set_xlabel('Dose [%]', fontsize=16)
ax.set_ylabel('Volume [%]', fontsize=16)
ax.set_xlim([0., 150.])
ax.set_ylim([0, 101])

ax.grid(True)
ax.legend();